In [1]:
from z3 import z3

In [2]:
puzzle = [
    [5, 3, 0,   0, 7, 0,   0, 0, 0],
    [6, 0, 0,   1, 9, 5,   0, 0, 0],
    [0, 9, 8,   0, 0, 0,   0, 6, 0],

    [8, 0, 0,   0, 6, 0,   0, 0, 3],
    [4, 0, 0,   8, 0, 3,   0, 0, 1],
    [7, 0, 0,   0, 2, 0,   0, 0, 6],

    [0, 6, 0,   0, 0, 0,   2, 8, 0],
    [0, 0, 0,   4, 1, 9,   0, 0, 5],
    [0, 0, 0,   0, 8, 0,   0, 7, 9],
]

# Lösen mit Z3

In [3]:
sol = z3.Solver()
felder = {}

## Spielregeln als Constraints

In [4]:
for zeile in range(1, 10):
    for spalte in range(1, 10):
        name = f"f{zeile}{spalte}"
        var = z3.Int(name)
        felder[name] = var
        sol.add(z3.And(1 <= var, var <= 9))

for zeile in range(1, 10):
    sol.add(z3.Distinct([felder[f"f{zeile}{spalte}"] for spalte in range(1, 10)]))

for spalte in range(1, 10):
    sol.add(z3.Distinct([felder[f"f{zeile}{spalte}"] for zeile in range(1, 10)]))

for block_zeile in [[1, 2, 3], [4, 5, 6], [7, 8, 9]]:
    for block_spalte in [[1, 2, 3], [4, 5, 6], [7, 8, 9]]:
        sol.add(z3.Distinct([felder[f"f{zeile}{spalte}"] for zeile in block_zeile for spalte in block_spalte]))

In [5]:
sol.check()

sat

## Vorhandene Einträge

In [6]:
# Add the initial puzzle clues
for zeile in range(9):
    for spalte in range(9):
        if puzzle[zeile][spalte] != 0:
            sol.add(felder[f"f{zeile+1}{spalte+1}"] == puzzle[zeile][spalte])

In [ ]:
sol.consequences([felder["f44"] == 7], [felder["f22"]])

## Lösen und Anzeigen

In [7]:
if sol.check() == z3.sat:    
    m = sol.model()
    solution = {}
    for name, var in felder.items():
        solution[name] = m.eval(var).as_long()

    for zeile in range(1, 10):
        for spalte in range(1, 10):
            value = solution[f"f{zeile}{spalte}"]
            print(f" {value:1} ", end="|" if spalte in [3, 6] else "")
        print("")
        if zeile in [3, 6]:
            print("---------+---------+---------")        
else:
    print("No soup for you!")

 5  3  4 | 6  7  8 | 9  1  2 
 6  7  2 | 1  9  5 | 3  4  8 
 1  9  8 | 3  4  2 | 5  6  7 
---------+---------+---------
 8  5  9 | 7  6  1 | 4  2  3 
 4  2  6 | 8  5  3 | 7  9  1 
 7  1  3 | 9  2  4 | 8  5  6 
---------+---------+---------
 9  6  1 | 5  3  7 | 2  8  4 
 2  8  7 | 4  1  9 | 6  3  5 
 3  4  5 | 2  8  6 | 1  7  9 
